In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pyautogui
import re
from bs4 import BeautifulSoup
import sys
sys.path.append('..')
import constants.constants as const
import constants.file_handler_constants as fh
from constants.accommodation_constants import *

from packages.accommodation.accommodation import *
from packages.file_handler_package.file_handler import *
from packages.accommodation.mulProcess_accommodation_scraping_proxy import *

import os
import glob
import time
import pandas as pd
import numpy as np
from dotenv import load_dotenv, dotenv_values 

from multiprocessing import Pool
from functools import partial

In [ ]:
# *** select one province from 'ALL_PROVINCE_ACCOMM_DATA'
# *** so, change "Idx_of_region" everytime when scrape another province
Idx_of_region = 0
cur_region_data = ALL_PROVINCE_ACCOMM_DATA[Idx_of_region]

# select first and last page to scrape
first_page = 444
last_page = 888

cur_province_en = cur_region_data[0]
cur_province_th = cur_region_data[1]
cur_province_url = cur_region_data[2]

# create list of : [1, 2, 3, ...., last_page]
page_number_list = np.arange(first_page, last_page + 1)
print("check page number list: ")
print(page_number_list)

# get list of dataframe result of all Accommodations in a province
# res_allAccommodations_list = []
with Pool(processes=5) as pool: 
    func_param_fix = partial(mulProcess_helper_scrape_accommodations_by_province, province_url = cur_province_url, province = cur_province_th)
    res_allAccommodations_list = pool.map(func_param_fix, page_number_list)
    print("end process for province --> %s " % (cur_province_th))

# concat all dataframe into a final dataframe to save in csv.
res_allAccommodations_df = pd.DataFrame()
print("check len of list result dataframe --> ", len(res_allAccommodations_list))
for Idx, cur_df in list(enumerate(res_allAccommodations_list)):
    print("cur_df --> page : ", Idx + first_page)
    print("len cur_df --> ", len(cur_df))
    res_allAccommodations_df = pd.concat([res_allAccommodations_df, cur_df])

# don't forget to remove row with lat/long be zero

# remove duplicate accommodation
res_allAccommodations_df.drop_duplicates(subset=['name'], inplace=True)
# set new index
res_allAccommodations_df.set_index(['name'], inplace=True)

# create directory to store result of scraping accommodation
# for example: 'accommodation_scraping\res_accommodation_scraping\res_accommodation_Phuket'
createDirectory(fh.STORE_ACCOMM_SCRAPING, os.path.join('res_accommodation_scraping', 'res_accommodation_%s' % (cur_province_en)))

# save result dataframe to .csv
# for example: 'res_accommodation_Phuket_page_1_44.csv'
res_file_name = 'res_accommodation_%s_page_%s_%s.csv' % (cur_province_en, first_page, last_page)
res_path = os.path.join(fh.STORE_ACCOMM_SCRAPING, 'res_accommodation_scraping', 'res_accommodation_%s' % (cur_province_en), res_file_name)
res_allAccommodations_df.to_csv(res_path, encoding="utf-8")
    

check page number list: 
[ 7  8  9 10 11 12]
end process for province --> ภูเก็ต 
check len of list result dataframe -->  6
cur_df --> page :  7
len cur_df -->  37
cur_df --> page :  8
len cur_df -->  37
cur_df --> page :  9
len cur_df -->  37
cur_df --> page :  10
len cur_df -->  37
cur_df --> page :  11
len cur_df -->  37
cur_df --> page :  12
len cur_df -->  37
Directory res_accommodation_scraping\res_accommodation_Phuket created successfully
